# Base methods to deal with the test cases

In [1]:
import pandas as pd
from pandas import DataFrame
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

### Base methods to deal with preprocessing data
- **fill_gaps** : *Method that deals with missing data from a dataframe*
- **normalize** : *Method that normalize data rescaling numerical values to a predefined range*
- **pca** : *Method that applies pca in a dataframe*

In [21]:
def fill_gaps(df, empty_token = '?'):
    imputer = SimpleImputer(missing_values = empty_token, strategy = 'most_frequent')
    result = imputer.fit_transform(df)
    
    result_df = DataFrame(result)
    result_df.columns = df.columns
    result_df.index = df.index
    return result_df

def normalize(df, columns, range=(0,1)):
    scaler = MinMaxScaler(feature_range=range)
    for col in columns:
        scaled_values = scaler.fit_transform(df[[col]].values.astype(float))
        df[col] = scaled_values
    return df

def pca(df, components = 70):
    pca = PCA(n_components = components)
    pca_values = pca.fit_transform(df)
    component_cols = []
    for i in range(0, components):
        component_cols.append('component_%s'%i)
    return DataFrame(data = pca_values, columns = component_cols)

def cost(fp, fn):
    return (fp * 2) + fn

def train_classifier(pre_train_df, pre_test_df):
    x_train = pre_train_df.iloc[:, 1:].values
    y_train = pre_train_df['class'].values

    x_test = pre_test_df.iloc[:, 1:].values
    y_test = pre_test_df['class'].values

    t1_classifier = DecisionTreeClassifier()
    t1_classifier.fit(x_train, y_train)

    y_pred = t1_classifier.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)
    display("Accuracy %s" % (accuracy))
    tn, fp, fn, tp = confusion_matrix(y_pred, y_test).ravel()
    display("Cost: %s" % (cost(fp, fn)))
    

# Load the train and test datasets

In [35]:
train_df = pd.read_csv("aps_training_set_sample3.csv")
test_df = pd.read_csv("aps_failure_test_set.csv")

# Balancing classes in train dataset by resampling

In [36]:
# we need to balance the classes 
positive_class = train_df[train_df['class'] == 'pos']
qtd_pos = positive_class.shape[0]
negative_class = train_df[train_df['class'] == 'neg']
qtd_neg = negative_class.shape[0]

display("Positive samples: %s - Negative samples: %s" %(qtd_pos, qtd_neg))
new_samples = min(qtd_neg, qtd_pos)
display("Using %s samples per class" % (new_samples))


train_df = pd.concat([positive_class[:new_samples], negative_class[:new_samples]], ignore_index = True)
display(train_df['class'].value_counts())
train_df.shape

'Positive samples: 1000 - Negative samples: 2091'

'Using 1000 samples per class'

neg    1000
pos    1000
Name: class, dtype: int64

(2000, 171)

# Test Case 1
- fill_gaps
- Using DecisionTreeClassifier from sklearn

In [37]:
pre_train_df = fill_gaps(train_df)
pre_test_df = fill_gaps(test_df)

display(pre_train_df['class'].value_counts())

train_classifier(pre_train_df, pre_test_df)

neg    1000
pos    1000
Name: class, dtype: int64

'Accuracy 0.9291875'

'Cost: 1154'

# Test Case 2
- fill_gaps
- normalizing top 10 std_dev columns

In [38]:
pre_train_df = fill_gaps(train_df)
# Ignoring first column that is the label
for c in pre_train_df.columns[1:].values:
    pre_train_df[c] = pd.to_numeric(pre_train_df[c])
    
std_df = pre_train_df.std(axis = 0, skipna=True)
top10_std = std_df.sort_values(ascending=[False])[:10]
display(top10_std)
top10_std_columns = top10_std.index
pre_train_df = normalize(pre_train_df, columns = top10_std_columns)

pre_test_df = fill_gaps(test_df)
pre_test_df = normalize(pre_test_df, columns = top10_std_columns)

train_classifier(pre_train_df, pre_test_df)

ac_000    5.943633e+08
dq_000    2.818990e+08
eb_000    1.113884e+08
bb_000    3.246790e+07
bv_000    3.235481e+07
cq_000    3.235481e+07
bu_000    3.235481e+07
bx_000    3.090717e+07
cc_000    2.912746e+07
du_000    2.895663e+07
dtype: float64

'Accuracy 0.921875'

'Cost: 1272'

# Test Case 3
- fill gaps
- normalizing top 10 std_dev columns
- dimensionality reduction through pca

In [43]:
pca_train_df = pca(pre_train_df.loc[:, pre_train_df.columns != 'class'])
pca_test_df = pca(pre_test_df.loc[:, pre_train_df.columns != 'class'])

pca_train_df = pd.concat([pre_train_df['class'], pca_train_df], axis=1)
pca_test_df = pd.concat([pre_test_df['class'], pca_test_df], axis = 1)

train_classifier(pca_train_df, pca_test_df)

'Accuracy 0.0240625'

'Cost: 15627'

In [44]:
pre_train_df.loc[:, pre_train_df.columns != 'class']

,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,ag_003,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,153204,0,8.541768e-08,0,0,0,0,0,0,11804,...,129862,26872,34044,22472,34362,0,0,0,0,0
1,453236,0,1.373253e-06,0,0,0,0,0,222,323436,...,7908038,3026002,5025350,2025766,1160638,533834,493800,6914,0,0
2,72504,0,7.481087e-07,1052,0,0,0,244,178226,1249396,...,1432098,372252,527514,358274,332818,284178,3742,0,0,0
3,762958,0,0.000000e+00,0,0,0,776,281128,2186308,8123016,...,0,0,0,0,0,0,0,0,0,0
4,695994,0,0.000000e+00,0,0,0,0,0,0,55620,...,1397742,495544,361646,28610,5130,212,0,0,0,0
5,1056758,0,1.986571e-05,856,0,0,0,50394,184552,2116260,...,1450086,713608,1750894,4054554,4096660,2295880,220478,482,0,0
6,361638,0,1.314118e-07,0,0,0,0,0,0,0,...,266410,114746,180006,108294,111218,264410,33734,0,0,0
7,791254,0,0.000000e+00,0,0,0,0,14868,921256,9292232,...,3625532,1380864,3262488,5727988,7563752,12114820,264714,198,0,0
8,1053152,0,0.000000e+00,0,0,0,2,2,30,192,...,7093206,3379622,7362340,8442584,8120144,5082396,6259760,176752,0,0
9,785270,0,0.000000e+00,8994,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
